In [19]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import sys
import os

In [20]:
#Need to specify fields from excel
source_filename = "907028 Dual Purpose DR_TFS Upload"
file_path = r"C:\\Users\\venkati.naidu\\hello\\Coursera\\PytonMaterial\\ML\\"

In [21]:
#Read the excel file from the project directory
excel_file = r'' + file_path + source_filename +'.xlsx'
final_excel = r'' + file_path + source_filename +'_merged.xlsx'
if not os.path.isfile(excel_file):
       print("File path {} does not exist. Exiting...".format(excel_file))
       sys.exit()
else:
    xlsx = pd.ExcelFile(excel_file)


In [22]:
"""
Columns to be included/validated in the report
"""
#usecols_mandatory=['Title','Area Path','Iteration Path','Initiative','Priority','Bundle']
usecols_mandatory=['Title','Area Path','Iteration Path','Initiative','Priority','Bundle','Application']
#usecols_mandatory=['Title','Area Path','Iteration Path','Initiative','Priority']
usecols_for_report=['Title', 'Area Path', 'Iteration Path','Priority', 'Action', 'Expected Result', 'Initiative','Bundle', 'Application']
#usecols_for_report=['Title', 'Area Path', 'Iteration Path','Priority', 'Action', 'Expected Result', 'Initiative']


In [23]:
"""
Below function is to verify the each field count is equal or not
"""

def check_tc_count(dfdict_param):
        tc_count = True
        old_value = 0;
        i = 0
        for key, value in dfdict_param.items(): 
         
            new_value = value
            if old_value == 0 and i == 0:
                i+=1
                old_value = new_value
            elif old_value == new_value:
                continue
            else:
                tc_count = False
                break
        return tc_count 
        


In [24]:
""" Find the lengthy columns if any (i.e. > 250 chars)"""

def found_lengthy_columns(use_case):
    df_lengthy_column = use_case[use_case["Title Length"]>256]
    if len(df_lengthy_column['Title'])>0:
        print("Title is too long:")
        print(df_lengthy_column['Title'],df_lengthy_column['Title Length'])
        #print ("{} {}".format(df_lengthy_column['Title'], df_lengthy_column['Title Length']))
        return True
    return False   


In [25]:
check_missed_mandatory_columns = []

def found_missing_mandatory_columns(sheet):
    from functools import reduce
    twolists=[]
    twolists.append(usecols_mandatory)
    twolists.append(df.columns.tolist())
    #print(f"Mandatory Columns:\n{usecols_mandatory}\n")
    #print(f"Columns from Sheet:{sheet}:\n{df.columns.tolist()}\n")
    #print(f"Both Mandatory and Columns from sheet:\n{twolists}\n")
    
    commonlist = list(reduce(lambda i, j: i & j, (set(x) for x in twolists)))
    #print(f"Common element extraction form N lists:\n{commonlist}" )
    missed_mandatory_columns = [i for i in usecols_mandatory if i not in commonlist]
    ##print(f"Missed Mandatory Columns in Excel Sheet:\n{missed_mandatory_columns}" )
    
    if len(missed_mandatory_columns) > 0:
        print(f"Missed Mandatory Columns in Excel Sheet: {sheet}\n{missed_mandatory_columns}" )
        check_missed_mandatory_columns.append(True)
        return True
    else:
        #print("No mismatches")
        return False
    
    

In [26]:
#boolean = not df["Student"].is_unique      # True (credit to @Carsten)
#boolean = df['Student'].duplicated().any() # True
def found_mismatch_area_iteration(use_case):
    list_areas = use_case["Area Path"].unique().tolist()
    list_iterations = use_case["Iteration Path"].unique().tolist()
    list_areas = [area for area in list_areas if str(area) != 'nan']
    list_iterations = [iteration for iteration in list_iterations if str(iteration) != 'nan']
    #print('list_areas',list_areas_count)
    #print('list_iteration',list_iterations_count)
    if len(list_areas)>1:
        print("Found Discrepancy in Area Path:")
        print("*"*15)
        for area in list_areas:
            print(area) 
        return True
    elif len(list_iterations)>1:
        print("Found Discrepancy in Iteration Path:")
        print("*"*15)
        for iteration in list_iterations:
            print(iteration) 
        return True
    return False   

#Report will be Printed

In [28]:
cannot_merge_excel = [] # list of xl sheets

all_sheets = pd.DataFrame()
for sheet in xlsx.sheet_names:

    df = xlsx.parse(sheet)
    #print(sheet)
    """
    Adding column names
    """
  
    
    if not found_missing_mandatory_columns(sheet):
        #print("Proceed with Testing")
    
        df_report = df[usecols_for_report]
        df = df[usecols_mandatory]
    
        """
        Counting the columns and storing the each field and respective row count in respective field
        """
        dfdict = dict(df.count())
        #print("print df",df.shape)
        #print(dfdict)
    
        """
        Following code will print the Usecase name (Sheet name) after field validation
        """
        df["Title Length"]= df["Title"].str.len()
        #p+=1
        
        if check_tc_count(dfdict): # and not found_mismatch_area_iteration(df):
            print('-' * 70)
            print(sheet," : PASS - Mandatory fields exist for all test cases")
            print("*"*15)
            print(dfdict)
            all_sheets =pd.concat([all_sheets, df_report],ignore_index=True)
            cannot_merge_excel.append(found_lengthy_columns(df)) # print test case title if it is too long
            cannot_merge_excel.append(found_mismatch_area_iteration(df))
            
            print('-' * 70)
            all_sheets.append(df_report, ignore_index=True)       
        else:
            """
            Print the fields and respective row count if the validation failed
            """
            print('-' * 70)
            print(sheet," : Test Case FIELD COUNT is NOT matching as below:")
            print('-' * 70)
            print ("{:<20} {:<20} ".format('Column Title', 'Row Count')) 
            for key, value in dfdict.items():
                print ("{:<25} {:<25} ".format(key, value))
            cannot_merge_excel.append(True)  ## if field count is not matching


if  cannot_merge_excel.count(True) > 0 or len(check_missed_mandatory_columns)>0 :
    print('#' * 70)
    print("Please fix the issue in Excel and come again")
    print('#' * 70)
else: # len(check_missed_mandatory_columns)>0:
    all_sheets.reindex()
    #Data Cleanup
    all_sheets = all_sheets.loc[:, ~all_sheets.columns.str.contains('^Unnamed')] # dropping unnamed columns
    #all_sheets = all_sheets.drop(all_sheets.columns[[0]], axis=1) # drop first 2 columns
    all_sheets = all_sheets.dropna(thresh=1) #Drop row if it does not have at least two values that are **not** NaN
    #Convert data frame to excel sheet
    all_sheets.to_excel(final_excel, index = False)
    all_sheets.head(10)
    print('#' * 70)
    print("Merged All Spread Sheets in a main Excel File:\n", final_excel)
    print('#' * 70)



----------------------------------------------------------------------
907028_Dual Purpose DR  : PASS - Mandatory fields exist for all test cases
***************
{'Title': 107, 'Area Path': 107, 'Iteration Path': 107, 'Initiative': 107, 'Priority': 107, 'Bundle': 107, 'Application': 107}
----------------------------------------------------------------------
######################################################################
Merged All Spread Sheets in a main Excel File:
 C:\\Users\\venkati.naidu\\hello\\Coursera\\PytonMaterial\\ML\\907028 Dual Purpose DR_TFS Upload_merged.xlsx
######################################################################


"""
This code will import all excel files from the current directory
ref:https://pythoninoffice.com/use-python-to-combine-multiple-excel-files/
import os
import pandas as pd
cwd = os.path.abspath('') 
files = os.listdir(cwd)  
df = pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        df = df.append(pd.read_excel(file), ignore_index=True) 
df.head() 
df.to_excel('total_sales.xlsx')
"""